# **Crawling PTA Trunojoyo (Manajemen)**

Crawling adalah proses otomatis dalam mengumpulkan data atau informasi dari internet menggunakan program yang disebut web crawler (atau bot). Web crawler bekerja dengan cara mengunjungi halaman-halaman web, membaca konten, lalu menyimpannya agar dapat diproses lebih lanjut.

Secara umum, web crawling adalah tahapan awal data mining di web.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re, sys, time
from concurrent.futures import ThreadPoolExecutor, as_completed


### **Tentukan Website yang Ingin di Crawling**

Disini saya ingin crawling dari website PTA Trunojoyo dan lebih spesifik hanya ke prodi manajemen

In [ ]:
BASE_URL = "https://pta.trunojoyo.ac.id/c_search/byprod"

## **Proses Crawling PTA**

Kita akan mengambil dari PTA khsusu untuk prodi manajemen dan hanya mengambil judul, penulis, pembimbing satu dan dua serta abstrak dalam bahasa indonesia

In [ ]:
def get_max_page(prodi_id):
    url = f"{BASE_URL}/{prodi_id}/1"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")

    last_page = soup.select_one('ol.pagination a:contains("»")')
    if last_page and "href" in last_page.attrs:
        href = last_page["href"]
        max_page = int(href.split("/")[-1])
        return max_page
    return 1

Fungsi `get_max_page(prodi_id)` dipakai untuk mencari tahu ada berapa halaman data (pagination) pada sebuah website untuk suatu program studi (`prodi_id`). Caranya: fungsi membuka halaman pertama, lalu mengecek apakah ada tombol menuju halaman terakhir (»). Jika ada, dia ambil angka terakhir dari link tersebut sebagai jumlah maksimal halaman. Kalau tidak ada tombol pagination, diasumsikan cuma ada 1 halaman.

1. Buat URL halaman pertama berdasarkan `BASE_URL` dan `prodi_id`.

2. Ambil isi halaman dengan` requests.get`.

3. Parsing HTML dengan BeautifulSoup.

4. Cari tombol pagination terakhir (yang ditandai dengan simbol `»`).

5. Kalau tombol ada → ambil nomor halaman terakhir dari link (`href`).

6. Kalau tidak ada → berarti hanya ada 1 halaman.

In [ ]:
def print_progress(current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\rPage {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n')

Fungsi `print_progress(current_page, total_pages)` digunakan untuk menampilkan progress bar di terminal agar kita bisa memantau sejauh mana proses sudah berjalan. Pertama, fungsi ini menghitung persentase berdasarkan halaman yang sedang diproses dibandingkan total halaman. Lalu, dibuat progress bar dengan panjang tetap (20 karakter), diisi dengan simbol blok `█` untuk bagian yang sudah selesai dan tanda `-` untuk bagian yang belum selesai. Setelah itu, progress bar ditampilkan di baris yang sama menggunakan `\r`, sehingga tampilannya selalu terupdate tanpa membuat baris baru. Terakhir, ketika proses mencapai halaman terakhir, fungsi menambahkan baris baru supaya tampilan rapi.

In [ ]:
def crawl_detail(link_keluar, prodi):
    try:
        id_match = re.search(r"/detail/(\d+)", link_keluar)
        pta_id = id_match.group(1) if id_match else None

        response = requests.get(link_keluar, timeout=10)
        soup1 = BeautifulSoup(response.content, "html.parser")
        isi = soup1.select_one('div#content_journal')

        judul = isi.select_one('a.title').text.strip()
        penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1]
        pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1]
        pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(' :')[1]

        paragraf = isi.select('p[align="justify"]')
        abstrak_id = paragraf[0].get_text(strip=True) if len(paragraf) > 0 else "N/A"
        abstrak_en = paragraf[1].get_text(strip=True) if len(paragraf) > 1 else "N/A"

        return {
            "id": pta_id,
            "penulis": penulis,
            "judul": judul,
            "abstrak_id": abstrak_id,
            "pembimbing_pertama": pembimbing_pertama,
            "pembimbing_kedua": pembimbing_kedua,
            "prodi": prodi,
        }
    except Exception:
        return None

def crawl_manajemen():
    start_time = time.time()
    results = []

    prodi_id = 7 # id prodi manajemen
    max_page = get_max_page(prodi_id)

    for j in range(1, max_page + 1):
        url = f"{BASE_URL}/{prodi_id}/{j}"
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")
        jurnals = soup.select('li[data-cat="#luxury"]')

        isii = soup.select_one('div#begin')
        if not isii:
            continue
        prodi_full = isii.select_one('h2').text.strip()
        prodi = prodi_full.replace("Journal Jurusan ", "")

        links = [jurnal.select_one('a.gray.button')['href'] for jurnal in jurnals]

        with ThreadPoolExecutor(max_workers=10) as executor:
            future_to_link = {executor.submit(crawl_detail, link, prodi): link for link in links}
            for future in as_completed(future_to_link):
                data = future.result()
                if data:
                    results.append(data)

        print_progress(j, max_page)

    df = pd.DataFrame(results)
    df.to_csv("pta_manajemen.csv", index=False, encoding="utf-8-sig")

    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    print("\n✅ Data Prodi Manajemen berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")

    return df

In [ ]:
crawl_manajemen()

/usr/local/lib/python3.12/dist-packages/soupsieve/css_parser.py:876: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


Page 207/207 [████████████████████] 100.00%

✅ Data Prodi Manajemen berhasil dikumpulkan!
📊 Total entri: 1031
⏱️ Waktu eksekusi: 0 jam 15 menit 0 detik


,id,penulis,judul,abstrak_id,pembimbing_pertama,pembimbing_kedua,prodi
0,080211100050,Wahyu Kurniawan,PENGARUH GAYA KEPEMIMPINAN DEMOKRATIK TERHADAP...,,"Dr. Dra. Hj. Iriani Ismail, MM","Dra. Hj. S. Anugrahini Irawati, MM",Manajemen
1,100211200002,Muhammad Zakaria Utomo,Pengukuran Website Quality Pada Situs Sistem A...,Aplikasi nyata pemanfaatan teknologi informasi...,"Dr. Ir. Nurita Andriani, MM","Nirma Kurriwati, SP, M.Si",Manajemen
2,080211100044,Hendri Wahyudi Prayitno,PENGARUH KEPEMIMPINAN DAN KOMPENSASI TERHADAP ...,Abstrak\r\nPenelitian ini menggunakan metode k...,"Dra. Hj. S Anugrahini Irawati, MM","Helmi Buyung Aulia,S,ST,SE,.MT",Manajemen
3,090211200001,Faishal,ANALISIS PERSEPSI BRAND ASSOCIATION MENURUT PE...,Tujuan penelitian ini adalah untuk mengetahui ...,Nurita Andriani,Yustina Chrismardani,Manajemen
4,080211100070,SATIYAH,PENGARUH FAKTOR-FAKTOR PELATIHAN DAN PENGEMBAN...,"ABSTRAK\r\nSatiyah, Pengaruh Faktor-faktor Pel...","Dra. Hj. S. Anugrahini Irawati, MM","Helmi Buyung Aulia,S,ST.SE,M.MT",Manajemen
...,...,...,...,...,...,...,...
1026,160211100291,Uswatun Hasanah,Pengaruh Pelatihan Dan Kompensasi Terhadap Pro...,"ABSTRAK\nUswatun Hasanah, 160211100291, Pengar...","Dr. Raden Mas Mochammad Wispandono S.E ., MS",,Manajemen
1027,160211100071,Husnul Hotimah,Analisis Cost Volume Profit Untuk Menentukan T...,ABSTRAK\nPenelitian ini bertujuan untuk menget...,"Hj. Evaliati Amaniyah, S.E., M.S.M.",,Manajemen
1028,160211100030,INTAN YULLIA NINGSIH,BAURAN PROMOSI PADA DEALER YAMAHA TRETAN MOTOR...,ABSTRAK\nPenelitian ini bertujuan: (1) Untuk m...,"DR. MOHAMMAD ARIEF, S.E., M.M.",,Manajemen
1029,160211100041,Novia Prihastiwi Ningtyas,"ANALISA PENGARUH RASIO BOPO, LDR, ROA TERHADAP...",Tujuan penelitian ini adalah untuk mengetahui ...,"Hj. Evaliati Amaniyah, S.E., M.S.M",,Manajemen


Fungsi `crawl_detail(link_keluar, prodi)` bertugas mengambil detail dari sebuah halaman jurnal. Pertama, fungsi mencoba mengekstrak ID PTA dari URL detail menggunakan regex. Kemudian, ia membuka halaman detail tersebut dengan `requests` dan mem-parsing isinya dengan BeautifulSoup. Dari halaman itu, fungsi mengambil informasi penting seperti judul, penulis, pembimbing I & II, serta abstrak bahasa Indonesia dan bahasa Inggris. Semua informasi tersebut dikembalikan dalam bentuk dictionary. Jika terjadi error, fungsi akan mengembalikan `None`.

Fungsi `crawl_manajemen()` dipakai khusus untuk meng-crawl data jurnal pada Prodi Manajemen. Pertama, fungsi mencatat waktu mulai agar nanti bisa dihitung lama eksekusinya. Setelah itu, ia menentukan jumlah halaman maksimal dengan `get_max_page`. Lalu, untuk setiap halaman, fungsi membuka daftar jurnal, mengambil nama lengkap prodi, lalu mengumpulkan semua link menuju detail jurnal. Link-link ini kemudian diproses secara paralel dengan `ThreadPoolExecutor` agar lebih cepat, memanggil `crawl_detail` untuk setiap link.

Hasil yang valid dikumpulkan dalam sebuah list, kemudian disimpan ke dalam CSV bernama `pta_manajemen.csv`. Setelah semua halaman selesai diproses, fungsi menghitung total entri, waktu eksekusi, dan menampilkannya dalam format jam-menit-detik. Pada akhirnya, fungsi mengembalikan data dalam bentuk DataFrame.

In [ ]:
from google.colab import files
files.download("/content/pta_manajemen.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Simpan dalam bentuk csv


## **Crawling Link**

Kali ini kita akan melakukan crawling link pada website labtia.trunojoyo. Tentukan link url daru website:

In [ ]:
BASE_URL = "https://labtia.trunojoyo.ac.id/"
START_URL = "https://labtia.trunojoyo.ac.id/"
headers = {"User-Agent": "Mozilla/5.0"}

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from collections import deque
import pandas as pd

visited = set()
edges = []  # simpan source -> target

def crawl(start_url, base_domain, max_pages=100, max_depth=3):
    queue = deque([(start_url, 0)])  # (url, depth)

    while queue and len(visited) < max_pages:
        url, depth = queue.popleft()

        if url in visited:
            continue
        visited.add(url)

        # stop kalau kedalaman melebihi max_depth
        if depth > max_depth:
            continue

        try:
            r = requests.get(url, timeout=10)
            if r.status_code != 200:
                continue
        except Exception as e:
            print(f"Gagal akses {url}: {e}")
            continue

        print(f"[{len(visited)}] Sedang di (depth {depth}): {url}")

        soup = BeautifulSoup(r.text, "html.parser")
        for link in soup.find_all("a", href=True):
            href = link['href']
            next_url = urljoin(url, href)
            parsed = urlparse(next_url)

            # hanya internal link
            if parsed.netloc == base_domain:
                # skip kalau masuk ke journals/*
                if "/journals/" in parsed.path.strip("/"):
                    continue

                # simpan relasi source -> target
                edges.append({
                    "source_url": url,
                    "target_url": next_url,
                    "depth": depth + 1
                })

                if next_url not in visited:
                    queue.append((next_url, depth + 1))


start_url = "https://labtia.trunojoyo.ac.id/"
base_domain = urlparse(start_url).netloc
crawl(start_url, base_domain, max_pages=20, max_depth=3)

# simpan ke CSV
df = pd.DataFrame(edges)
df.to_csv("hasil_crawl_edges.csv", index=False, encoding="utf-8")

print("\nTotal halaman unik:", len(visited))
print("Relasi source→target disimpan di 'hasil_crawl_edges.csv'")


[1] Sedang di (depth 0): https://labtia.trunojoyo.ac.id/
[2] Sedang di (depth 1): http://labtia.trunojoyo.ac.id/
[4] Sedang di (depth 1): http://labtia.trunojoyo.ac.id/p/about-us.html
[5] Sedang di (depth 1): https://labtia.trunojoyo.ac.id/search
[6] Sedang di (depth 1): https://labtia.trunojoyo.ac.id/#!
[7] Sedang di (depth 1): https://labtia.trunojoyo.ac.id/p/contact.html
[8] Sedang di (depth 1): https://labtia.trunojoyo.ac.id/search/label/PRAKTIKUM
[9] Sedang di (depth 1): https://labtia.trunojoyo.ac.id/search/label/Publikasi
[10] Sedang di (depth 1): https://labtia.trunojoyo.ac.id/2025/07/pengumuman-calon-wargalab-jurusan.html
[11] Sedang di (depth 1): https://labtia.trunojoyo.ac.id/2024/09/jadwal-pratikum-semester-ganjil-tahun.html
[12] Sedang di (depth 1): https://labtia.trunojoyo.ac.id/2025/09/jadwal-pratikum-semester-ganjil-tahun.html
[13] Sedang di (depth 1): https://labtia.trunojoyo.ac.id/2024/07/pengumuman-calon-warga-lab-jurusan.html
[14] Sedang di (depth 1): https://labtia

In [ ]:
df

,source_url,target_url,depth
0,https://labtia.trunojoyo.ac.id/,http://labtia.trunojoyo.ac.id/,1
1,https://labtia.trunojoyo.ac.id/,https://labtia.trunojoyo.ac.id/mega,1
2,https://labtia.trunojoyo.ac.id/,https://labtia.trunojoyo.ac.id/mega,1
3,https://labtia.trunojoyo.ac.id/,https://labtia.trunojoyo.ac.id/mega,1
4,https://labtia.trunojoyo.ac.id/,https://labtia.trunojoyo.ac.id/,1
...,...,...,...
1569,https://labtia.trunojoyo.ac.id/p/sitemap.html,https://labtia.trunojoyo.ac.id/search/label/Pu...,2
1570,https://labtia.trunojoyo.ac.id/p/sitemap.html,https://labtia.trunojoyo.ac.id/search/label/Tech,2
1571,https://labtia.trunojoyo.ac.id/p/sitemap.html,https://labtia.trunojoyo.ac.id/p/contact.html,2
1572,https://labtia.trunojoyo.ac.id/p/sitemap.html,https://labtia.trunojoyo.ac.id/,2


In [ ]:
from google.colab import files
files.download("/content/hasil_crawl_edges.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

simpan dalam bentuk csv, untuk digunakan dalam data processing

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from tabulate import tabulate

kategori_list = ["nasional", "bisnis", "tekno", "bola", "metro", "dunia"]
base_rss = "https://rss.tempo.co"

results = []

for kategori in kategori_list:
    rss_url = f"{base_rss}/{kategori}"
    print(f"Mengambil RSS dari: {rss_url}")
    resp = requests.get(rss_url, headers={"User-Agent": "Mozilla/5.0"})

    if resp.status_code != 200:
        print(f"Gagal mengambil RSS dari kategori: {kategori}")
        continue

    soup = BeautifulSoup(resp.content, "xml")
    items = soup.find_all("item")

    for item in items[:20]:
        link = item.find("link").text
        judul = item.find("title").text.strip()

        try:
            berita_id = link.split("/")[-1].split("-")[0]
        except:
            berita_id = None

        try:
            r = requests.get(link, headers={"User-Agent": "Mozilla/5.0"})
            if r.status_code != 200:
                print(f"Gagal membuka link: {link}")
                continue

            s = BeautifulSoup(r.text, "html.parser")

            isi_tag = s.find("div", id="content-wrapper")
            if isi_tag:
                for tag in isi_tag(["script", "style", "aside", "figure"]):
                    tag.decompose()

                isi_paragraf = isi_tag.find_all("p")
                isi = " ".join(p.get_text(" ", strip=True) for p in isi_paragraf)
                isi = re.sub(r"Baca juga.*?(?=[A-Z])", "", isi)

            else:
                isi = None

        except Exception as e:
            print(f"Error saat mengambil isi dari {link}: {e}")
            isi = None

        isi_ringkas = (isi[:200] + "...") if isi else "-"

        results.append([len(results)+1, berita_id, judul, kategori, isi_ringkas])
        time.sleep(1)

df = pd.DataFrame(results, columns=["No", "ID Berita", "Judul", "Kategori", "Isi"])

df.to_csv("berita_tempo.csv", index=False, encoding="utf-8-sig")

print(tabulate(df, headers="keys", tablefmt="grid", showindex=False))


Mengambil RSS dari: https://rss.tempo.co/nasional
Mengambil RSS dari: https://rss.tempo.co/bisnis
Mengambil RSS dari: https://rss.tempo.co/tekno
Mengambil RSS dari: https://rss.tempo.co/bola
Mengambil RSS dari: https://rss.tempo.co/metro
Mengambil RSS dari: https://rss.tempo.co/dunia
Gagal mengambil RSS dari kategori: dunia
+------+-------------+-------------------------------------------------------------------------------------------------------+------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|   No | ID Berita   | Judul                                                                                                 | Kategori   | Isi                                                                                                                                                                                         

# Penjelasan Kode Scraping RSS Tempo

Kode ini bertujuan untuk **mengambil berita dari RSS Tempo** berdasarkan beberapa kategori, lalu menyimpannya ke file **CSV**.  

---

## Alur Kerja Program

1. **Menentukan daftar kategori**  
   Daftar kategori berita sudah ditentukan:  
   - nasional  
   - bisnis  
   - tekno  
   - bola  
   - metro  
   - dunia  

   Dari tiap kategori, program membentuk URL RSS, misalnya:  

2. **Mengambil RSS feed**  
- Program mengirim request menggunakan `requests`.  
- Jika request gagal → kategori dilewati.  
- Jika berhasil → konten diproses dengan **BeautifulSoup** (mode XML) untuk menemukan semua elemen `<item>` yang berisi berita.  

3. **Membatasi jumlah berita**  
Dari setiap kategori, hanya **20 berita pertama** yang akan diproses.  

4. **Mengambil informasi berita**  
Dari setiap `<item>`, program mengambil:  
- **Link berita** dari `<link>`  
- **Judul berita** dari `<title>`  
- **ID berita**, dengan cara memotong bagian URL  

5. **Membuka halaman berita asli**  
- Program membuka link berita dengan `requests`.  
- Jika halaman berhasil dibuka → konten berita diambil dari `<div id="content-wrapper">`.  
- Elemen yang tidak relevan seperti `<script>`, `<style>`, `<aside>`, dan `<figure>` dihapus.  
- Semua paragraf `<p>` digabung jadi satu teks panjang.  
- Bagian teks **“Baca juga…”** dihapus menggunakan regex.  

6. **Ringkasan isi berita**  
- Jika isi berhasil didapat, dibuat ringkasan berupa **200 karakter pertama**.  
- Jika gagal, isi diganti dengan tanda `-`.  

7. **Menyimpan hasil**  
- Setiap berita dicatat dalam list `results` dengan format:  
  ```
  [No, ID Berita, Judul, Kategori, Isi Ringkas]
  ```  
- Untuk menghindari beban server, ada jeda `time.sleep(1)` setiap kali memproses berita.  

8. **Membuat DataFrame dan CSV**  
- Semua hasil dikonversi ke **Pandas DataFrame** dengan kolom:  
  ```
  No | ID Berita | Judul | Kategori | Isi
  ```  
- DataFrame disimpan ke file **berita_tempo.csv** dengan encoding UTF-8.  

---

## Hasil Akhir
- File `berita_tempo.csv` berisi daftar berita dari berbagai kategori.  
- Isi berita ditampilkan dalam bentuk ringkasan agar tidak terlalu panjang.  


In [ ]:
from google.colab import files
files.download("/content/berita_tempo.csv")

Simpan dalam bentuk csv